In [1]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

data=pd.read_excel('北京市空气质量数据.xlsx')
data=data.replace(0,np.NaN)
data['年']=data['日期'].apply(lambda x:x.year)
month=data['日期'].apply(lambda x:x.month)
quarter_month={'1':'一季度','2':'一季度','3':'一季度',
               '4':'二季度','5':'二季度','6':'二季度',
               '7':'三季度','8':'三季度','9':'三季度',
              '10':'四季度','11':'四季度','12':'四季度'}
data['季度']=month.map(lambda x:quarter_month[str(x)])
bins=[0,50,100,150,200,300,1000]
data['等级']=pd.cut(data['AQI'],bins,labels=['一级优','二级良','三级轻度污染','四级中度污染','五级重度污染','六级严重污染'])
print('对AQI的分组结果：\n{0}'.format(data[['日期','AQI','等级','季度']]))

对AQI的分组结果：
             日期    AQI      等级   季度
0    2014-01-01   81.0     二级良  一季度
1    2014-01-02  145.0  三级轻度污染  一季度
2    2014-01-03   74.0     二级良  一季度
3    2014-01-04  149.0  三级轻度污染  一季度
4    2014-01-05  119.0  三级轻度污染  一季度
...         ...    ...     ...  ...
2150 2019-11-22  183.0  四级中度污染  四季度
2151 2019-11-23  175.0  四级中度污染  四季度
2152 2019-11-24   30.0     一级优  四季度
2153 2019-11-25   40.0     一级优  四季度
2154 2019-11-26   73.0     二级良  四季度

[2155 rows x 4 columns]


In [2]:
from collections import Iterable
isinstance(month,Iterable)   #判断month是否为可迭代的对象


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  """Entry point for launching an IPython kernel.


True

代码说明：
（1）第6行：利用数据框函数replace()将数据框中的0（表示无监测结果）替换为缺失值NaN。
（2）第7，8行：利用.apply()方法以及匿名函数，基于“日期”变量得到每个样本观测的年份和月份。
（3）第9行：建立一个关于月份和季度的字典quarter_month。
（4）第10行：利用Python函数map()，依据字典quarter_month，将序列month中的1，2，3等月份映射（对应）到相应的季度上。
（5）第14行：生成一个后续用于对AQI分组的列表bins。它描述了AQI和空气质量等级的数值对应关系。
（6）第15行：利用Pandas的cut()方法对AQI进行分组。

In [3]:
print('各季度AQI和PM2.5的均值:\n{0}'.format(data.loc[:,['AQI','PM2.5']].groupby(data['季度']).mean()))
print('各季度AQI和PM2.5的描述统计量:\n',data.groupby(data['季度'])['AQI','PM2.5'].apply(lambda x:x.describe()))

def top(df,n=10,column='AQI'):
    return df.sort_values(by=column,ascending=False)[:n]
print('空气质量最差的5天:\n',top(data,n=5)[['日期','AQI','PM2.5','等级']])
print('各季度空气质量最差的3天:\n',data.groupby(data['季度']).apply(lambda x:top(x,n=3)[['日期','AQI','PM2.5','等级']]))
print('各季度空气质量情况:\n',pd.crosstab(data['等级'],data['季度'],margins=True,margins_name='总计',normalize=False))

各季度AQI和PM2.5的均值:
            AQI      PM2.5
季度                        
一季度  109.327778  77.225926
三季度   98.911071  49.528131
二季度  109.369004  55.149723
四季度  109.612403  77.195736
各季度AQI和PM2.5的描述统计量:
                   AQI       PM2.5
季度                               
一季度 count  540.000000  540.000000
    mean   109.327778   77.225926
    std     80.405408   73.133857
    min     26.000000    4.000000
    25%     48.000000   24.000000
    50%     80.000000   53.000000
    75%    145.000000  109.250000
    max    470.000000  454.000000
三季度 count  551.000000  551.000000
    mean    98.911071   49.528131
    std     45.484516   35.394897
    min     28.000000    3.000000
    25%     60.000000   23.000000
    50%     95.000000   41.000000
    75%    130.500000   67.000000
    max    252.000000  202.000000
二季度 count  542.000000  541.000000
    mean   109.369004   55.149723
    std     49.608042   35.918345
    min     35.000000    5.000000
    25%     71.000000   27.000000
    50%     99.000

代码说明：
（1）第1行：利用数据框的groupby()方法，计算各季度AQI和PM2.5的平均值。
（2）第2行：计算几个季度AQI和PM2.5的基本描述统计量（均值，标准差，最小值，四分位数，最大值）。
（3）第4，5行：定义了一个名为top的用户自定义函数：对给定数据框，按指定列（默认AQI列）值的降序排序，返回排在前n（默认10）条数据。
（4）第6行：调用用户自定义函数top，对data数据框中，按AQI值的降序排序并返回前5条数据，即AQI最高的5天的数据。
（5）第7行：首先对数据按季度分组，依次对分组数据调用用户自定义函数top，得到各季度AQI最高的3天数据。
（6）第8行：利用Pandas函数crosstab()对数据按季度和空气质量等级交叉分组，并给出各个组的样本量。

In [4]:
pd.get_dummies(data['等级'])
data.join(pd.get_dummies(data['等级']))

,日期,AQI,质量等级,PM2.5,PM10,SO2,CO,NO2,O3,年,季度,等级,一级优,二级良,三级轻度污染,四级中度污染,五级重度污染,六级严重污染
0,2014-01-01,81.0,良,45.0,111.0,28.0,1.5,62.0,52.0,2014,一季度,二级良,0,1,0,0,0,0
1,2014-01-02,145.0,轻度污染,111.0,168.0,69.0,3.4,93.0,14.0,2014,一季度,三级轻度污染,0,0,1,0,0,0
2,2014-01-03,74.0,良,47.0,98.0,29.0,1.3,52.0,56.0,2014,一季度,二级良,0,1,0,0,0,0
3,2014-01-04,149.0,轻度污染,114.0,147.0,40.0,2.8,75.0,14.0,2014,一季度,三级轻度污染,0,0,1,0,0,0
4,2014-01-05,119.0,轻度污染,91.0,117.0,36.0,2.3,67.0,44.0,2014,一季度,三级轻度污染,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2150,2019-11-22,183.0,中度污染,138.0,181.0,9.0,2.4,94.0,5.0,2019,四季度,四级中度污染,0,0,0,1,0,0
2151,2019-11-23,175.0,中度污染,132.0,137.0,6.0,1.6,69.0,34.0,2019,四季度,四级中度污染,0,0,0,1,0,0
2152,2019-11-24,30.0,优,7.0,30.0,3.0,0.2,11.0,58.0,2019,四季度,一级优,1,0,0,0,0,0
2153,2019-11-25,40.0,优,13.0,30.0,3.0,0.4,32.0,29.0,2019,四季度,一级优,1,0,0,0,0,0


代码说明：
（1）第1行：利用Pandas的get_dummies得到分类型变量“等级”的哑变量。
（2）第2行：利用数据框的join()方法，将原始数据和哑变量数据，按行索引进行横向合并。

In [5]:
np.random.seed(123)
sampler=np.random.randint(0,len(data),10)
print(sampler)
sampler=np.random.permutation(len(data))[:10]
print(sampler)

data.take(sampler)
data.loc[data['质量等级']=='优',:]

[1346 1122 1766 2154 1147 1593 1761   96   47   73]
[1883  326   43 1627 1750 1440  993 1469 1892  865]


,日期,AQI,质量等级,PM2.5,PM10,SO2,CO,NO2,O3,年,季度,等级
7,2014-01-08,27.0,优,15.0,25.0,13.0,0.5,21.0,53.0,2014,一季度,一级优
8,2014-01-09,46.0,优,27.0,46.0,19.0,0.8,35.0,53.0,2014,一季度,一级优
11,2014-01-12,47.0,优,27.0,47.0,27.0,0.7,39.0,59.0,2014,一季度,一级优
19,2014-01-20,35.0,优,8.0,35.0,6.0,0.3,15.0,65.0,2014,一季度,一级优
20,2014-01-21,26.0,优,18.0,25.0,27.0,0.7,34.0,50.0,2014,一季度,一级优
...,...,...,...,...,...,...,...,...,...,...,...,...
2122,2019-10-25,30.0,优,8.0,20.0,2.0,0.4,24.0,55.0,2019,四季度,一级优
2131,2019-11-03,48.0,优,33.0,48.0,3.0,0.6,34.0,33.0,2019,四季度,一级优
2135,2019-11-07,47.0,优,24.0,47.0,3.0,0.5,37.0,44.0,2019,四季度,一级优
2152,2019-11-24,30.0,优,7.0,30.0,3.0,0.2,11.0,58.0,2019,四季度,一级优


代码说明：
（1）第2行：利用Pandas函数random.randint()在指定范围内随机抽取指定个数（这里是10）的随机数。
（2）第 4行：利用Pandas函数random.permutation是对数据随机打乱重排。之后再抽取前10个样本观测。
（3）第7行：利用数据框的take()方法，基于指定随机数获得数据集的一个子集。
（4）第8行：利用数据框访问的方式，抽取满足指定条件（质量等级等于优）行的数据。
